### Introduccion
Implementar un modelo de ML para clasificar si un dominio es legítimo o es Phishing. El phising se basa en la ingeniería social (manipulación de emociones, aprovechamiento de atajos mentales y sesgos cognitivos) para engañar a las víctimas y lograr que estas den información (normalmente credenciales).

### Parte I --- Ingeniería de características

### Explotación de datos

1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.

In [3]:
import pandas as pd

data = pd.read_csv('data/dataset_phishing.csv')
data.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como “pishing”. ¿Está balanceado el dataset?

In [4]:
print(data['status'].value_counts())

status
legitimate    5715
phishing      5715
Name: count, dtype: int64


### Derivación de características

1. ¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo de vida del dominio, o las características de la página Web?

- El análisis de una URL tiene varias ventajas importantes frente a otros tipos de análisis. En primer lugar, es mucho más rápido y ligero, ya que no requiere cargar ni inspeccionar el contenido completo de una página web. Esto permite detectar phishing en tiempo real, incluso antes de que el usuario acceda al sitio. Además, el análisis de URL no depende de servicios externos ni del historial del dominio, lo que lo hace más independiente y resistente ante ataques nuevos. También es más seguro, porque evita interactuar directamente con páginas que pueden ser maliciosas.


2. ¿Qué características de una URL son más prometedoras para la detección de phishing?

- Las características más prometedoras de una URL para detectar phishing son aquellas relacionadas con su estructura y composición. Entre las más útiles se encuentran la longitud de la URL, la cantidad de subdominios, el uso de direcciones IP en lugar de nombres de dominio, la presencia de caracteres especiales, y patrones sospechosos como guiones o múltiples puntos. También resultan muy relevantes las características léxicas y estadísticas, como la frecuencia de símbolos no alfanuméricos o la entropía de estos caracteres. Estas propiedades suelen diferenciar claramente una URL legítima de una maliciosa y permiten a los modelos de machine learning identificar phishing con alta precisión.